# Phaser 3 bolas documentacion

## Descripción

Este proyecto es un juego desarrollado con Phaser 3 que incluye balas que rebotan y un jugador que puede moverse manualmente o automáticamente utilizando una red neuronal.

## Dependencias

* Phaser 3
* Synaptic.js

## Configuración

* Incluye las bibliotecas de Phaser y Synaptic en tu archivo HTML.* 
Precarga las imágenes y sprites necesarios.* 
Inicializa el juego y la física de Phase
* Configura el jugador, la bala y el fondo.do.

 # Código

### index.html

In [ ]:
<!doctype html>
<html>
<head>
    <meta charset="UTF-8" />
    <title>Phaser 3 Balas</title>
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            overflow: hidden;
            background: #000;
            flex-direction: column;
        }
    </style>
    <script src="lib/phaser.min.js"></script>
    <script src="lib/synaptic.js"></script>
</head>
<body>
    <h1 style="color: aliceblue; display: flex;">Game Phaser 3 bolas</h1>
    <div class="juego" style="display: flex; -ms-flex-align: center; ">
        <script src="demo1.js" type="text/javascript"></script>
    </div>
    <h2 style="color: aliceblue; display: flex;">Josué Montalván Zavala -- 20120137</h2>

    <button onclick="redirectToAnotherPage()" style="width: 200px; height: 50px;">Juego pelota rebotes</button>
    <br>
    <br>

    <!-- Función JavaScript para redireccionar -->
    <script>
        function redirectToAnotherPage() {
            // Cambia la URL a la que deseas redireccionar
            window.location.href = 'rebotes.html';
        }
    </script>
</body>
</html>


### rebotes.html

In [ ]:
<!doctype html>
<html>
<head>
    <meta charset="UTF-8" />
    <title>Phaser 3 Balas</title>
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            overflow: hidden;
            background: #000;
            flex-direction: column;
        }
    </style>
    <script src="lib/phaser.min.js"></script>
    <script src="lib/synaptic.js"></script>
</head>
<body>
    <h1 style="color: aliceblue; display: flex;"> Juego rebotes</h1>
    <div class="juego" style="display: flex; -ms-flex-align: center; ">
        <script src="phaser2.js" type="text/javascript"></script>
    </div>
    <h2 style="color: aliceblue; display: flex;">Josué Montalván Zavala -- 20120137</h2>

    <button onclick="redirectToAnotherPage()" style="width: 200px; height: 50px;">Juego phaser 3 naves </button>
    <br>
    <br>
    <!-- Función JavaScript para redireccionar -->
    <script>
        function redirectToAnotherPage() {
            // Cambia la URL a la que deseas redireccionar
            window.location.href = 'index.html';
        }
    </script>
</body>
</html>


## JavaScript

### demo1.js variables globales

In [ ]:
var w = 350;
var h = 350;
var jugador;
var fondo;
var bala;
var VOLVIENDOV = false;
var VOLVIENDOH = false;
var cursors;
var menu;
var estatusIzquierda;
var estatusDerecha;
var estatusArriba;
var estatusAbajo;
var estatusMovimiento;
var nnNetwork, nnEntrenamiento, nnSalida, datosEntrenamiento = [];
var modoAuto = false, eCompleto = false;
var JX = 200;
var JY = 200;
var autoMode = false;
var juego = new Phaser.Game(w, h, Phaser.CANVAS, '', { preload: preload, create: create, update: update, render: render });


### Precargar recursos

In [ ]:
function preload() {
    juego.load.image('background', 'assets/game/background1.jpg');
    juego.load.spritesheet('mono', 'assets/sprites/altair.png', 32, 48);
    juego.load.image('menu', 'assets/game/menu.png');
    juego.load.image('bala', 'assets/sprites/purple_ball.png');
}


### Crear elementos del juego

In [ ]:
function create() {
    juego.physics.startSystem(Phaser.Physics.ARCADE);
    juego.physics.arcade.gravity.y = 0;
    juego.time.desiredFps = 30;

    fondo = juego.add.tileSprite(0, 0, w, h, 'background');
    jugador = juego.add.sprite(w / 2, h / 2, 'mono');

    juego.physics.enable(jugador);
    jugador.body.collideWorldBounds = true;
    var corre = jugador.animations.add('corre', [8, 9, 10, 11]);
    jugador.animations.play('corre', 10, true);

    bala = juego.add.sprite(0, 0, 'bala');
    juego.physics.enable(bala);
    bala.body.collideWorldBounds = true;
    bala.body.bounce.set(1);
    setRandomBalaVelocity();

    pausaL = juego.add.text(w - 100, 20, 'Pausa', { font: '20px Arial', fill: '#fff' });
    pausaL.inputEnabled = true;
    pausaL.events.onInputUp.add(pausa, self);
    juego.input.onDown.add(mPausa, self);

    cursors = juego.input.keyboard.createCursorKeys();

    nnNetwork = new synaptic.Architect.Perceptron(3, 6, 6, 6, 5);
    nnEntrenamiento = new synaptic.Trainer(nnNetwork);
}


### Funciones de la red neuronal

In [ ]:
function enRedNeural() {
    nnEntrenamiento.train(datosEntrenamiento, { rate: 0.0003, iterations: 10000, shuffle: true });
}

function datosVertical(param_entrada) {
    nnSalida = nnNetwork.activate(param_entrada);
    return nnSalida[2] >= nnSalida[3];
}

function datosHorizontal(param_entrada) {
    nnSalida = nnNetwork.activate(param_entrada);
    return nnSalida[0] >= nnSalida[1];
}

function datosMovimiento(param_entrada) {
    nnSalida = nnNetwork.activate(param_entrada);
    return nnSalida[4] * 100 >= 20;
}


### Funciones del juego

In [ ]:
function pausa() {
    juego.paused = true;
    menu = juego.add.sprite(w / 2, h / 2, 'menu');
    menu.anchor.setTo(0.5, 0.5);
}

function mPausa(event) {
    if (juego.paused) {
        var menu_x1 = w / 2 - 270 / 2, menu_x2 = w / 2 + 270 / 2,
            menu_y1 = h / 2 - 180 / 2, menu_y2 = h / 2 + 180 / 2;
        var mouse_x = event.x,
            mouse_y = event.y;
        if (mouse_x > menu_x1 && mouse_x < menu_x2 && mouse_y > menu_y1 && mouse_y < menu_y2) {
            if (mouse_x >= menu_x1 && mouse_x <= menu_x2 && mouse_y >= menu_y1 && mouse_y <= menu_y1 + 90) {
                eCompleto = false;
                datosEntrenamiento = [];
                modoAuto = false;
            } else if (mouse_x >= menu_x1 && mouse_x <= menu_x2 && mouse_y >= menu_y1 + 90 && mouse_y <= menu_y2) {
                if (!eCompleto) {
                    enRedNeural();
                    eCompleto = true;
                }
                modoAuto = true;
            }
            menu.destroy();
            reset();
            juego.paused = false;
        } else {
            menu.destroy();
            juego.paused = false;
        }
    }
}

function reset() {
    JX = 200;
    JY = 200;
    jugador.reset(JX, JY);
}

function update() {
    if (modoAuto) {
        // Lógica de movimiento automático
    } else {
        if (cursors.left.isDown) {
            jugador.body.velocity.x = -150;
        }
        if (cursors.right.isDown) {
            jugador.body.velocity.x = 150;
        }
        if (cursors.up.isDown) {
            jugador.body.velocity.y = -150;
        }
        if (cursors.down.isDown) {
            jugador.body.velocity.y = 150;
        }
    }

    if (!modoAuto && !cursors.left.isDown && !cursors.right.isDown && !cursors.up.isDown && !cursors.down.isDown) {
        jugador.body.velocity.x = 0;
        jugador.body.velocity.y = 0;
    }
}

function render() {
    juego.debug.text('Modo Auto: ' + (modoAuto ? 'Activado' : 'Desactivado'), 20, 30);
    juego.debug.text('Entrenamiento: ' + datosEntrenamiento.length + ' valores', 20, 50);
}

function setRandomBalaVelocity() {
    var speed = 300;
    var angle = juego.rnd.angle();
    bala.body.velocity.set(Math.cos(angle) * speed, Math.sin(angle) * speed);
}

function colisionH() {
    autoMode = true;
    pausa();
}


# Explicación del Código

### Variables Globales

* w y h definen el ancho y alto del juego.
* jugador es el sprite del jugador.
* fondo es el fondo del juego.
* bala es el sprite de la bala.
* VOLVIENDOV y VOLVIENDOH indican si el jugador está volviendo al centro.
* cursors almacena las teclas de dirección.
* nnNetwork es la red neuronal.
* datosEntrenamiento almacena los datos de entrenamiento.
* modoAuto y eCompleto indican si el modo automático está activado y si el entrenamiento está completo.

## Funciones



##### `preload()`
Carga las imágenes y sprites.

##### `create()`
Inicializa el juego, la física, y configura el jugador, la bala y la red neuronal.

##### `enRedNeural()`
Entrena la red neuronal.

##### `datosVertical()`, `datosHorizontal()`, y `datosMovimiento()`
Activan la red neuronal y determinan las acciones del jugador.

##### `pausa()` y `mPausa()`
Manejan el estado de pausa del juego.

##### `reset()`
Reinicia la posición del jugador.

##### `update()`
Maneja el movimiento del jugador.

##### `render()`
Dibuja el texto de depuración en la pantalla.

##### `setRandomBalaVelocity()`
Establece una velocidad aleatoria para la bala.
idad aleatoria para la bala.
la bala.

# Créditos

### Josué Montalván Zavala -- 20120137




